In [ ]:
import keras
from keras import layers
import matplotlib.pyplot as plt
import cv2 as cv
import pandas as pd
import os
from keras.models import Model
from keras import layers, Input
import faiss
import numpy as np

input_img = keras.Input(shape=(512, 512, 1))

# Encoder
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(4, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = layers.Conv2D(4, (3, 3), activation='relu', padding='same')(encoded)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
path = '/kaggle/working'

x_train = []
y_train = []

for i in os.listdir(os.path.join(path, 'queries')):
  x_train.append(cv.imread(os.path.join(path, 'queries', i), cv.IMREAD_GRAYSCALE))

for i in os.listdir(os.path.join(path, 'database_2D')):
  y_train.append(cv.imread(os.path.join(path, 'database_2D', i), cv.IMREAD_GRAYSCALE))

x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = x_train.astype('float32') / 255.
y_train = y_train.astype('float32') / 255.

x_train = np.reshape(x_train, (len(x_train), 512, 512, 1))
y_train = np.reshape(y_train, (len(y_train), 512, 512, 1))

x_test = x_train
y_test = y_train

In [ ]:
autoencoder.fit(x_train, y_train,
                epochs=50,
                batch_size=64,
                shuffle=True,
                validation_data=(x_test, y_test))

In [ ]:
decoded_imgs = autoencoder.predict(x_test)

n = 10
plt.figure(figsize=(20, 4))
for i in range(1, n + 1):
    # Display original
    ax = plt.subplot(2, n, i)
    plt.imshow(x_test[i].reshape(512, 512))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + n)
    plt.imshow(decoded_imgs[i].reshape(512, 512))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
def extract_features(image_path, model):
    img = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
    img = np.reshape(img, (1, 512, 512, 1))
    encoder = Model(model.input, model.layers[6].output)
    features = encoder.predict(img)
    return features.flatten()

# Load and extract features for training images
training_path = 'database_2D'
query_path = 'queries'
query_mapping = {}

training_features = []
training_files = []

for file in os.listdir(training_path):
    file_path = os.path.join(training_path, file)
    features = extract_features(file_path, autoencoder)
    training_features.append(features)
    training_files.append(file)

# Flatten the list of descriptors and create an index
training_features = np.vstack(training_features).astype(np.float32)
index = faiss.IndexFlatL2(training_features.shape[1])
index.add(training_features)

# Function to find top-k matches
def find_top_k_matches(features, k=5):
    features = features.astype(np.float32)
    D, I = index.search(np.expand_dims(features, axis=0), k)
    return I[0]

mrr_at_5 = 0

for query in os.listdir(query_path):
    query_obj = query.split('.')[0]
    query_path_full = os.path.join(query_path, query)
    query_features = extract_features(query_path_full, autoencoder)

    # Find top-5 matches
    indices = find_top_k_matches(query_features, k=5)

    scores = [(i, training_files[i].split('.')[0] + '.stl') for i in indices]

    query_mapping[query] = '"'
    for i in range(5):
        query_mapping[query] += scores[i][1] + ','
        if scores[i][1].split('.')[0] == query_obj:
            mrr_at_5 += 1 / (i + 1)
    query_mapping[query] = query_mapping[query][:-1] + '"'

    print(query, query_mapping[query])

queries_len = len(os.listdir(query_path))
print('MRR@5: ', mrr_at_5 / queries_len)

In [ ]:
output = {
    'query': query_mapping.keys(),
    'label': query_mapping.values()
}

df = pd.DataFrame.from_dict(output)
df.to_csv('predict.csv', sep=',', index=False)